In [1]:
import mne
from mne.coreg import Coregistration
from mne.minimum_norm import make_inverse_operator, apply_inverse_epochs
import numpy as np
from pathlib import Path
import pandas as pd
import os
from pathlib import Path
from mne_connectivity import spectral_connectivity_epochs
import xarray as xr
import time
from spectral_connectivity import Multitaper, Connectivity
import gc

In [2]:
def filter_labels_with_vertices(labels_parc, src):
    # Get the vertices from both hemispheres in the source space
    src_vertices = [src[0]['vertno'], src[1]['vertno']]
    
    # Initialize an empty list to hold valid labels
    valid_labels = []
    
    for label in labels_parc:
        # Determine the hemisphere index: 0 for 'lh' and 1 for 'rh'
        hemi_idx = 0 if label.hemi == 'lh' else 1
        
        # Check if any of the label's vertices are in the source space for that hemisphere
        if any(v in src_vertices[hemi_idx] for v in label.vertices):
            valid_labels.append(label)
            
    return valid_labels

In [33]:
subjects_dir = "/Users/lina_01/Desktop/repos/Connectivity_EEG_benchmarking/inspected"
#subjects_dir = "/scratch/MPI-LEMON/freesurfer/subjects/inspected"
#subjects = ["sub-032302"]
subjects = [subject for subject in os.listdir(subjects_dir) if subject.startswith('sub')]
#subjects_complete = [subject.split("_")[0] for subject in os.listdir("imcoh_results")]
#remaining_subjects = [subject for subject in subjects if subject not in subjects_complete]
conditions = ['EC','EO']

In [5]:
start_time = time.time()

for subject in remaining_subjects:
    try:
        conditions_mats = []
        for condition in conditions:
            src = mne.setup_source_space(subject, add_dist="patch", subjects_dir=subjects_dir)
            #raw = mne.io.read_raw_eeglab(f"/Users/lina_01/Desktop/eeg/{subject}/{subject}_{condition}.set")
            raw = mne.io.read_raw_eeglab(f"/scratch/MPI-LEMON/freesurfer/subjects/eeg/{subject}/{subject}_{condition}.set")
            
            info = raw.info
            fiducials = "estimated"
            coreg = Coregistration(info, subject, subjects_dir, fiducials=fiducials)
            
            conductivity = (0.3, 0.006, 0.3)
            model = mne.make_bem_model(subject=subject, conductivity=conductivity, subjects_dir=subjects_dir)
            bem = mne.make_bem_solution(model)
            
            epochs = mne.make_fixed_length_epochs(raw, duration=1.0, preload=False)
            epochs.set_eeg_reference(projection=True)
            epochs.apply_baseline((None,None))
            fwd = mne.make_forward_solution(
                epochs.info, trans=coreg.trans, src=src, bem=bem, verbose=True
            )
            
            cov = mne.compute_covariance(epochs)
            
            inv = mne.minimum_norm.make_inverse_operator(epochs.info, fwd, cov, verbose=True)
            
            method = "sLORETA"
            snr = 3.0
            lambda2 = 1.0 / snr**2
            stc = apply_inverse_epochs(
                epochs,
                inv,
                lambda2,
                method=method,
                pick_ori=None,
                verbose=True,
                return_generator=False
            )
            
            labels_parc = mne.read_labels_from_annot(subject, parc='aparc', subjects_dir=subjects_dir)
            
            filtered_labels = filter_labels_with_vertices(labels_parc, src)
            label_ts = mne.extract_label_time_course(stc, filtered_labels, src, mode='auto', return_generator=False, allow_empty=False)
            
            n=len(epochs)
    
            mats=[]
            
            for i in range(100):
                inds = np.random.choice(range(n),int(n/2),replace=False)
                m = Multitaper(time_series=np.array(label_ts)[inds].transpose(2, 0, 1),
                    sampling_frequency=250)
                
                c = Connectivity(fourier_coefficients=m.fft(),
                        frequencies=m.frequencies,
                        time=m.time)    
                
                con = getattr(c, 'generalized_partial_directed_coherence')()
                mat = np.nan_to_num(con.squeeze())
                mats.append(mat[8:14,:,:])
                frequencies = c.frequencies
                del m
                del c
                del con
                gc.collect()
    
            region = [label.name for label in filtered_labels]
            frequencies = list(frequencies[8:14])
            bootstrap_samples = list(range(100))
    
            xarray = xr.DataArray(np.array(mats), dims=["bootstrap_samples","frequencies","region1","region2"],
                                  coords={"bootstrap_samples":bootstrap_samples,"frequencies":frequencies,"region1":region, "region2":region})
            xarray.to_netcdf(f'imcoh_results/{subject}_array_imcoh_{condition}.nc')
            
    except Exception as e:
        print(f'failed on {subject}')
        print(e)
        continue

print(time.time()-start_time)

Setting up the source space with the following parameters:

SUBJECTS_DIR = /Users/lina_01/Desktop
Subject      = sub-032302
Surface      = white
Octahedron subdivision grade 6

>>> 1. Creating the source space...

Doing the octahedral vertex picking...
Loading /Users/lina_01/Desktop/sub-032302/surf/lh.white...
Mapping lh sub-032302 -> oct (6) ...
    Triangle neighbors and vertex normals...
Loading geometry from /Users/lina_01/Desktop/sub-032302/surf/lh.sphere...
Setting up the triangulation for the decimated surface...
loaded lh.white 4098/125183 selected to source space (oct = 6)

Loading /Users/lina_01/Desktop/sub-032302/surf/rh.white...
Mapping rh sub-032302 -> oct (6) ...
    Triangle neighbors and vertex normals...
Loading geometry from /Users/lina_01/Desktop/sub-032302/surf/rh.sphere...
Setting up the triangulation for the decimated surface...
loaded rh.white 4098/126579 selected to source space (oct = 6)

Calculating patch information (limit=0.0 mm)...
    Computing patch stati

/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_1581/3649947725.py:9: RuntimeWarning: Data file name in EEG.data (sub-010003_EC.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-032302_EC.fdt).
  raw = mne.io.read_raw_eeglab(f"/Users/lina_01/Desktop/eeg/{subject}/{subject}_{condition}.set")
/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_1581/3649947725.py:9: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(f"/Users/lina_01/Desktop/eeg/{subject}/{subject}_{condition}.set")
/var/folders/cl/8cj5mvt51nbchvsgng871t400000gn/T/ipykernel_1581/3649947725.py:9: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(f"/Users/lina_01/Desktop/eeg/{subject}/{subject}_{condition}.set")


Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
outer skin  CM is   0.81 -10.23  24.94 mm
outer skull CM is   0.80 -10.13  24.44 mm
inner skull CM is   0.79 -12.40  34.37 mm
Checking that surface outer skull is inside surface outer skin  ...
Checking that surface inner skull is inside surface outer skull ...
Checking distance between outer skin  and outer skull surfaces...
Minimum distance between the outer skin  and outer skull surfaces is approximately    1.8 mm
Checking distance between outer skull and inner skull surfaces...
Minimum distance between the outer skull and inner skull surfaces is approximately    1.2 mm
Surfaces passed the basic topology checks.
Complete.

Three-layer model surfaces loaded.
Computing the linear collocation solution...
    Matrix coefficients...
        outer skin  (2562) -> outer skin  (2562) ...
        outer skin  (2562) -> outer skull (2562) ...
   

Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum iterations reached. 0 of 1 converged
Maximum it

KeyboardInterrupt: 